In [3]:
!pip install gensim

In [1]:
!pip install --upgrade --no-deps --force-reinstall numpy
!pip install --upgrade --force-reinstall gensim

  Using cached numpy-2.3.2-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (62 kB)
Using cached numpy-2.3.2-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (16.6 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4


  Using cached gensim-4.3.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (8.1 kB)
  Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached scipy-1.13.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
Using cached gensim-4.3.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (26.6 MB)
Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.0 MB)
Using cached scipy-1.13.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (38.2 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 kB 8.5 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.17.3
    Uninstalling wrapt-1.17.3:
      Successfully uninstalled wrapt-1.17.3
  Attempting uninstall: numpy
    Found existing installation: numpy 2.3.2
    Uninstalling num

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report
import numpy as np

In [4]:
dataset = pd.read_csv('reddit_preprocessing.csv')
cleaned_dataset = dataset.dropna()

X_cleaned = cleaned_dataset['clean_comment']
y_cleaned = cleaned_dataset['category']

In [5]:
# Split the cleaned data into train and test sets (80-20 split)
X_train_cleaned, X_test_cleaned, y_train_cleaned, y_test_cleaned = train_test_split(X_cleaned, y_cleaned, test_size=0.2, random_state=42)

In [6]:
X_train_tokenized = [sentence.split() for sentence in X_train_cleaned]
X_test_tokenized= [sentence.split() for sentence in X_test_cleaned]

In [7]:
word2vec_model = Word2Vec(sentences = X_train_tokenized, vector_size=300, window=5, min_count=1, workers=4, sg=1)

In [8]:
# Generate the vector representation for each comment
def vectorize_comments(tokenized_comments, word2vec_model):
    vectorized_comments = []
    for tokens in tokenized_comments:
        vectors = [word2vec_model.wv[token] for token in tokens if token in word2vec_model.wv]
        if len(vectors) > 0:
            vectorized_comments.append(np.mean(vectors, axis=0))  # Mean of all word vectors in the comment
        else:
            vectorized_comments.append(np.zeros(word2vec_model.vector_size))  # If no words match, use a zero vector
    return np.array(vectorized_comments)

In [9]:
# Vectorize train and test comments
X_train_word2vec = vectorize_comments(X_train_tokenized, word2vec_model)
X_test_word2vec = vectorize_comments(X_test_tokenized, word2vec_model)

In [10]:
X_train_word2vec.shape

(29329, 300)

In [14]:
# Define the LightGBM model with your best settings
best_model = LGBMClassifier(
    objective='multiclass',
    num_class=3,
    metric="multi_logloss",
    is_unbalance=True,
    class_weight="balanced",
    reg_alpha=0.1,  # L1 regularization
    reg_lambda=0.1,  # L2 regularization,
    learning_rate=0.08081298097796712,
    n_estimators=479,
    max_depth=19
)

In [15]:
# Train the LightGBM model on Word2Vec embeddings
best_model.fit(X_train_word2vec, y_train_cleaned)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.109885 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 76500
[LightGBM] [Info] Number of data points in the train set: 29329, number of used features: 300
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612


LGBMClassifier(class_weight='balanced', is_unbalance=True,
               learning_rate=0.08081298097796712, max_depth=19,
               metric='multi_logloss', n_estimators=479, num_class=3,
               objective='multiclass', reg_alpha=0.1, reg_lambda=0.1)

In [16]:
# Make predictions on the test data
y_pred = best_model.predict(X_test_word2vec)

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [17]:
print(classification_report(y_test_cleaned, y_pred))

              precision    recall  f1-score   support

          -1       0.49      0.47      0.48      1647
           0       0.73      0.73      0.73      2510
           1       0.69      0.70      0.69      3176

    accuracy                           0.66      7333
   macro avg       0.64      0.63      0.63      7333
weighted avg       0.66      0.66      0.66      7333



In [18]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 24.5 MB/s eta 0:00:00


In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report
import numpy as np
import optuna
from sklearn.model_selection import cross_val_score

# Load the dataset
dataset = pd.read_csv('reddit_preprocessing.csv')

# Drop rows with NaN values in 'clean_comment'
cleaned_dataset = dataset.dropna()

# Separate features and target
X_cleaned = cleaned_dataset['clean_comment']
y_cleaned = cleaned_dataset['category']

# Split the cleaned data into train and test sets (80-20 split)
X_train_cleaned, X_test_cleaned, y_train_cleaned, y_test_cleaned = train_test_split(
    X_cleaned, y_cleaned, test_size=0.2, random_state=42)

# Tokenize the sentences for Word2Vec training
X_train_tokenized = [sentence.split() for sentence in X_train_cleaned]
X_test_tokenized = [sentence.split() for sentence in X_test_cleaned]

# Train Word2Vec model
word2vec_model = Word2Vec(sentences=X_train_tokenized, vector_size=100, window=5, min_count=1, workers=4, sg=1)  # sg=1 for Skip-Gram model

# Generate the vector representation for each comment
def vectorize_comments(tokenized_comments, word2vec_model):
    vectorized_comments = []
    for tokens in tokenized_comments:
        vectors = [word2vec_model.wv[token] for token in tokens if token in word2vec_model.wv]
        if len(vectors) > 0:
            vectorized_comments.append(np.mean(vectors, axis=0))  # Mean of all word vectors in the comment
        else:
            vectorized_comments.append(np.zeros(word2vec_model.vector_size))  # If no words match, use a zero vector
    return np.array(vectorized_comments)

# Vectorize train and test comments
X_train_word2vec = vectorize_comments(X_train_tokenized, word2vec_model)
X_test_word2vec = vectorize_comments(X_test_tokenized, word2vec_model)

# Encode the target labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train_cleaned)
y_test_encoded = label_encoder.transform(y_test_cleaned)

# Objective function for Optuna hyperparameter tuning
def objective(trial):
    # Suggest hyperparameters to be tuned
    params = {
        "objective": "multiclass",
        "num_class": 3,
        "metric": "multi_logloss",
        "is_unbalance": True,
        "class_weight": "balanced",
        "reg_alpha": 0.1,  # L1 regularization
        "reg_lambda": 0.1,  # L2 regularization
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "max_depth": trial.suggest_int("max_depth", 3, 20)
    }

    # Initialize the LightGBM model with suggested parameters
    model = LGBMClassifier(**params)

    # Perform cross-validation to evaluate the model performance
    scores = cross_val_score(model, X_train_word2vec, y_train_encoded, cv=3, scoring='accuracy')

    # Return the mean accuracy score across folds
    return scores.mean()

# Create an Optuna study to optimize the objective function
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=30)  # Run 30 trials

# Print the best trial
print("Best trial:")
best_params = study.best_trial.params
print(best_params)

# Train the LightGBM model with the best parameters
best_model = LGBMClassifier(
    objective='multiclass',
    num_class=3,
    metric="multi_logloss",
    is_unbalance=True,
    class_weight="balanced",
    reg_alpha=0.1,
    reg_lambda=0.1,
    **best_params
)

best_model.fit(X_train_word2vec, y_train_encoded)

# Make predictions on the test data
y_pred = best_model.predict(X_test_word2vec)




[I 2025-08-22 14:52:06,042] A new study created in memory with name: no-name-da01ea75-0846-4c17-9edc-4a2557471e40


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.022668 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 25500
[LightGBM] [Info] Number of data points in the train set: 19552, number of used features: 100
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.024180 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 25500
[LightGBM] [Info] Number of data points in the train set: 19553, number of used features: 100
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


...

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.043152 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 25500
[LightGBM] [Info] Number of data points in the train set: 19553, number of used features: 100
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
[I 2025-08-22 16:09:29,348] Trial 29 finished with value: 0.6523237634603541 and parameters: {'n_estimators': 836, 'learning_rate': 0.1906143286841374, 'max_depth': 10}. Best is trial 26 with value: 0.6535851542033279.


Best trial:
{'n_estimators': 920, 'learning_rate': 0.22568846989646146, 'max_depth': 18}
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.040240 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 25500
[LightGBM] [Info] Number of data points in the train set: 29329, number of used features: 100
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [20]:
# Print classification report
print(classification_report(y_test_encoded, y_pred))

              precision    recall  f1-score   support

           0       0.50      0.39      0.44      1647
           1       0.73      0.72      0.72      2510
           2       0.66      0.75      0.70      3176

    accuracy                           0.66      7333
   macro avg       0.63      0.62      0.62      7333
weighted avg       0.65      0.66      0.65      7333

